In [1]:
print("Initialize libraries")
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation,normalization
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.optimizers import SGD
from keras.layers.advanced_activations import PReLU

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler, MaxAbsScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.callbacks import ModelCheckpoint
from keras.models import  Model
from keras.layers import Input,concatenate,Dense
import keras.backend as K
from keras import activations
from keras.engine.topology import Layer, InputSpec
import gc
import os
import warnings
warnings.filterwarnings("ignore")
import keras.backend.tensorflow_backend as KTF
from scipy.sparse import csr_matrix, hstack, coo_matrix
import tensorflow as tf
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

KTF.set_session(sess)

Initialize libraries


Using TensorFlow backend.


# Basic Features

In [3]:
%%time
age_test = pd.read_csv('../age_test.csv', header = None, names = ['uId'])
age_train = pd.read_csv('../age_train.csv', header = None, names = ['uId','age_group'])
data = pd.concat([age_train,age_test], axis = 0,sort=True).reset_index()
data.drop(['index'],axis=1,inplace=True)
del age_test, age_train
gc.collect()

CPU times: user 502 ms, sys: 143 ms, total: 645 ms
Wall time: 643 ms


In [4]:
user_basic_info = pd.read_csv('../user_basic_info.csv',header= None, names=['uId','gender','city','prodName','ramCapacity','ramLeftRation','romCapacity','romLeftRation','color','fontSize','ct','carrier','os'])
user_behavior_info = pd.read_csv('../user_behavior_info.csv', header = None, names =  ['uId','bootTimes','AFuncTimes','BFuncTimes','CFuncTimes','DFuncTimes','EFuncTimes','FFuncTimes','FFuncSum'])
data = data.merge(user_basic_info).merge(user_behavior_info)
del user_basic_info, user_behavior_info
gc.collect()

70

In [5]:
%%time
for i in ['A','B','C','D','E','F']:
    data['{}FuncTimes'.format(i)] = round(abs(data['{}FuncTimes'.format(i)]))
    
data['ramLeftCapacity'] = data['ramCapacity'] * data['ramLeftRation']
data['romLeftCapacity'] = data['romCapacity'] * data['romLeftRation']
data['romLeftRation'][data.romLeftRation>1] = 1
data['ramLeftRation'][data.ramLeftRation>1] = 1

CPU times: user 496 ms, sys: 155 ms, total: 651 ms
Wall time: 627 ms


In [5]:
%%time
#usage表中top5000使用量的appId
app_usage5000 = pd.read_hdf('./feature/app_usage5000.h5', key='data')
user_app_actived = pd.read_csv('./data/user_app_actived.csv', header = None, names =['uId','appId'])
app_usage5000.columns=['uId', 'usage_appId']
data = data.merge(user_app_actived, on='uId', how='left').merge(app_usage5000, how='left', on='uId')
X_app = CountVectorizer(token_pattern='a\d+',binary=True).fit_transform(data['appId'])
X_usage = CountVectorizer(token_pattern='a\d+', binary=True).fit_transform(data['usage_appId'].fillna('None'))
del user_app_actived
del app_usage5000
gc.collect()

CPU times: user 2min 45s, sys: 10.5 s, total: 2min 55s
Wall time: 3min 1s


In [8]:
%%time
app_usage5000 = pd.read_hdf('../feature/app_usage5000_top_times.h5', key='data')
app_usage5000.columns=['uId', 'usage_appId']
data = data.merge(app_usage5000, how='left', on='uId')
X_usage = CountVectorizer(token_pattern='a\d+', binary=True).fit_transform(data['usage_appId'].fillna('None'))
del app_usage5000
gc.collect()

CPU times: user 50.7 s, sys: 3.55 s, total: 54.3 s
Wall time: 54.5 s


In [11]:
cate_feat = [ 'gender', 'city', 'prodName', 'color', 'fontSize', 'ct', 'carrier', 'os']
num_feat = ['ramLeftCapacity','romLeftCapacity', 'ramCapacity', 'ramLeftRation', 'romCapacity', 'romLeftRation', 'bootTimes',
            'AFuncTimes', 'BFuncTimes', 'CFuncTimes', 'DFuncTimes', 'EFuncTimes', 'FFuncTimes', 'FFuncSum']

X_num = RobustScaler().fit_transform(data[num_feat].fillna(0))

for feat in cate_feat:
     data[feat] = LabelEncoder().fit_transform(data[feat].fillna('None').apply(str))

X_cate = OneHotEncoder().fit_transform(data[cate_feat].fillna(-1))
#X1 = hstack((X_app, X_cate, X_num), format='csr')
X2 = hstack((X_usage, X_cate, X_num), format='csr')
del X_cate, X_num
gc.collect()

57

In [12]:
test_index = list(data[np.isnan(data.age_group)].index)
train_index = list(data[~np.isnan(data.age_group)].index)
# train_x_app = X1[train_index]
# test_x_app  = X1[test_index]
train_x_usage = X2[train_index]
test_x_usage  = X2[test_index]
print('All features: train shape {}, test shape {}'.format(train_x_usage.shape, test_x_usage.shape))

All features: train shape (2010000, 11767), test shape (502500, 11767)


In [14]:
targetencoder = LabelEncoder().fit(data[~np.isnan(data.age_group)].age_group)
y = targetencoder.transform(data[~np.isnan(data.age_group)].age_group)
nclasses = len(targetencoder.classes_)
dummy_y = np_utils.to_categorical(y)

#X_train_app, X_val_app, y_train_app, y_val_app = train_test_split(train_x_app, dummy_y, test_size=0.02, random_state=42)
X_train_usage, X_val_usage, y_train_usage, y_val_usage = train_test_split(train_x_usage, dummy_y, test_size=0.02, random_state=42)
#del train_x_app, train_x_usage, dummy_y
#gc.collect()

In [15]:
def batch_generator(X, y, batch_size, shuffle):
    #chenglong code for fiting from generator (https://www.kaggle.com/c/talkingdata-mobile-user-demographics/forums/t/22567/neural-network-for-sparse-matrices)
    number_of_batches = np.ceil(X.shape[0]/batch_size)
    counter = 0
    sample_index = np.arange(X.shape[0])
    if shuffle:
        np.random.shuffle(sample_index)
    while True:
        batch_index = sample_index[batch_size*counter:batch_size*(counter+1)]
        X_batch = X[batch_index,:].toarray()
        y_batch = y[batch_index]
        counter += 1
        yield X_batch, y_batch
        if (counter == number_of_batches):
            if shuffle:
                np.random.shuffle(sample_index)
            counter = 0

def batch_generatorp(X, batch_size, shuffle):
    number_of_batches = X.shape[0] / np.ceil(X.shape[0]/batch_size)
    counter = 0
    sample_index = np.arange(X.shape[0])
    while True:
        batch_index = sample_index[batch_size * counter:batch_size * (counter + 1)]
        X_batch = X[batch_index, :].toarray()
        counter += 1
        yield X_batch
        if (counter == number_of_batches):
            counter = 0

# 定义DNN+FM模型

In [11]:
class FM(Layer): 
    def __init__(self, output_dim=30, activation="relu",**kwargs): 
        self.output_dim = output_dim 
        self.activate = activations.get(activation) 
        super(FM, self).__init__(**kwargs) 
        
    def build(self, input_shape): 
        self.weight = self.add_weight(name='weight',shape=(input_shape[1], self.output_dim),initializer='glorot_uniform',trainable=True) 
        self.bias = self.add_weight(name='bias',shape=(self.output_dim,),initializer='zeros',trainable=True) 
        self.kernel = self.add_weight(name='kernel',shape=(input_shape[1], self.output_dim),initializer='glorot_uniform',trainable=True) 
        super(FM, self).build(input_shape) 
        
    def call(self, x):
        feature = K.dot(x,self.weight) + self.bias
        a = K.pow(K.dot(x,self.kernel), 2)
        b = K.dot(x, K.pow(self.kernel, 2))
        cross = K.mean(a-b, 1, keepdims=True)*0.5
        cross = K.repeat_elements(K.reshape(cross, (-1, 1)), self.output_dim, axis=-1) 
        return self.activate(feature + cross) 
    
    def compute_output_shape(self, input_shape): 
        return (input_shape[0], self.output_dim)

In [16]:
def baseline_model_app():
    # create two models
    input1  = Input(shape=(X_train_app.shape[1],))

    dense_1 = Dense(32, kernel_initializer='normal', activation='relu')(input1)
    #dense_1 = Dropout(0.6)(dense_1)
    dense_2 = Dense(16, kernel_initializer='normal', activation='relu')(dense_1)
    #dense_2 = Dropout(0.6)(dense_2)
    dense_3 = Dense(8, kernel_initializer='normal', activation='relu')(dense_2)
    #dense_3 = Dropout(0.6)(dense_3)
    out     = Dense(6,kernel_initializer='normal', activation='softmax')(dense_3)
    # Compile model
    model = Model(inputs=input1, outputs = out)
    model.compile(loss ='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])  #logloss
    return model

In [16]:
def baseline_model_usage():
    # create two models
    input1  = Input(shape=(X_train_usage.shape[1],))

    dense_1 = Dense(32, kernel_initializer='normal', activation='relu')(input1)
    #dense_1 = Dropout(0.6)(dense_1)
    dense_2 = Dense(16, kernel_initializer='normal', activation='relu')(dense_1)
    #dense_2 = Dropout(0.6)(dense_2)
    dense_3 = Dense(8, kernel_initializer='normal', activation='relu')(dense_2)
    #dense_3 = Dropout(0.6)(dense_3)
    out     = Dense(6,kernel_initializer='normal', activation='softmax')(dense_3)
    # Compile model
    model = Model(inputs=input1, outputs = out)
    model.compile(loss ='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])  #logloss
    return model

In [13]:
def DenseNet():
    # create two models
    input1  = Input(shape=(X_train.shape[1],))

    dense_1 = Dense(256, kernel_initializer='normal', activation='relu')(input1)
    #dense_1 = Dropout(0.6)(dense_1)
    dense_2 = Dense(128, kernel_initializer='normal', activation='relu')(dense_1)
    #dense_2 = Dropout(0.6)(dense_2)
    dense_2_x  = concatenate([dense_1,dense_2])
    dense_3 = Dense(64, kernel_initializer='normal', activation='relu')(dense_2_x)
    #dense_3 = Dropout(0.6)(dense_3)
    dense_3_x  = concatenate([dense_1,dense_2,dense_3])
    dense_4 = Dense(7, kernel_initializer='normal', activation='relu')(dense_3_x)
    #dense_4 = Dropout(0.6)(dense_4)
    dense_4_x  = concatenate([dense_1,dense_2,dense_3,dense_4])
    out     = Dense(6,kernel_initializer='normal', activation='softmax')(dense_4_x)
    # Compile model
    model = Model(inputs=input1, outputs = out)
    model.compile(loss ='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])  #logloss
    return model

In [14]:
def FMNet():
    # create two models
    input1  = Input(shape=(X_train.shape[1],))
    #DNN_model_I
    dense_1 = Dense(100, kernel_initializer='normal', activation='tanh')(input1)
    dense_2 = Dense(150, kernel_initializer='normal', activation='tanh')(dense_1)
    dense_3 = Dense(150, kernel_initializer='normal', activation='tanh')(dense_2)
    dense_4 = Dense(100, kernel_initializer='normal', activation='tanh')(dense_3)
    dense_5 = Dense(64, kernel_initializer='normal', activation='tanh')(dense_4)
    #FM_model_II
    FM_1    = FM(200)(input1)
    FM_2    = FM(64)(FM_1)

    x       = concatenate([dense_5,FM_2])
    x_tmp   = Dense(32,kernel_initializer='normal', activation='softmax')(x)
    out     = Dense(6,kernel_initializer='normal', activation='softmax')(x_tmp)
    # Compile model
    model = Model(inputs=input1, outputs = out)
    model.compile(loss ='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])  #logloss
    #model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])  #logloss
    return model

In [17]:
model=baseline_model_usage()

In [ ]:
checkpoint = ModelCheckpoint(filepath='./weight_bias/best_epoch_model_app_usage_top_times.hdf5',monitor='val_acc',mode='max' ,save_best_only=True,verbose=1,period=1)
 
callback_lists=[checkpoint]


fit= model.fit_generator(generator=batch_generator(X_train_usage, y_train_usage, 1024, True),
                         nb_epoch=20,
                         samples_per_epoch=np.ceil(X_train_usage.shape[0]/1024) ,
                         validation_data=(X_val_usage.todense(), y_val_usage), verbose=1,
                         shuffle=True,
                         callbacks=callback_lists
                         )

Epoch 1/20
1924/1924 [==============================] - 207s 108ms/step - loss: 1.1642 - acc: 0.4993 - val_loss: 1.0818 - val_acc: 0.5401

Epoch 00001: val_acc improved from -inf to 0.54015, saving model to ./weight_bias/best_epoch_model_app_usage_top_times.hdf5
Epoch 2/20
1924/1924 [==============================] - 207s 107ms/step - loss: 1.0578 - acc: 0.5495 - val_loss: 1.0543 - val_acc: 0.5529

Epoch 00002: val_acc improved from 0.54015 to 0.55294, saving model to ./weight_bias/best_epoch_model_app_usage_top_times.hdf5
Epoch 3/20
1924/1924 [==============================] - 207s 107ms/step - loss: 1.0388 - acc: 0.5583 - val_loss: 1.0440 - val_acc: 0.5570

Epoch 00003: val_acc improved from 0.55294 to 0.55697, saving model to ./weight_bias/best_epoch_model_app_usage_top_times.hdf5
Epoch 4/20
1924/1924 [==============================] - 207s 107ms/step - loss: 1.0285 - acc: 0.5628 - val_loss: 1.0403 - val_acc: 0.5601

Epoch 00004: val_acc improved from 0.55697 to 0.56012, saving mode

In [20]:
model=baseline_model_usage()

In [21]:
checkpoint = ModelCheckpoint(filepath='./weight_bias/best_epoch_model_usage.hdf5',monitor='val_acc',mode='max' ,save_best_only=True,verbose=1,period=1)
 
callback_lists=[checkpoint]


fit= model.fit_generator(generator=batch_generator(X_train_usage, y_train_usage, 1024, True),
                         nb_epoch=20,
                         samples_per_epoch=np.ceil(X_train_usage.shape[0]/1024) ,
                         validation_data=(X_val_usage.todense(), y_val_usage), verbose=1,
                         shuffle=True,
                         callbacks=callback_lists 
                         )

Epoch 1/1
1924/1924 [==============================] - 233s 121ms/step - loss: 1.1621 - acc: 0.4998 - val_loss: 1.0832 - val_acc: 0.5392

Epoch 00001: val_acc improved from -inf to 0.53923, saving model to ./weight_bias/best_epoch_model_usage.hdf5


In [ ]:
from keras.models import load_model
#保存X_app权重
model = load_model('./weight_bias/best_epoch_model_app.hdf5')
weight = model.get_weights()[0]
bias = model.get_weights()[1]
weight = weight[:X_app.shape[1],:]
np.save('./weight_bias/X_app_weight_dense1.npy',weight)
np.save('./weight_bias/X_app_bias_dense1.npy',bias)
weight = model.get_weights()[2]
bias = model.get_weights()[3]
np.save('./weight_bias/X_app_weight_dense2.npy',weight)
np.save('./weight_bias/X_app_bias_dense2.npy',bias)
weight = model.get_weights()[4]
bias = model.get_weights()[5]
np.save('./weight_bias/X_app_weight_dense3.npy',weight)
np.save('./weight_bias/X_app_bias_dense3.npy',bias)

In [23]:
from keras.models import load_model
#保存X_usage权重
model = load_model('./weight_bias/best_epoch_model_app_usage_top_times.hdf5')
weight = model.get_weights()[0]
bias = model.get_weights()[1]
weight = weight[:X_usage.shape[1],:]
np.save('./weight_bias/X_usage_weight_times_dense1.npy',weight)
np.save('./weight_bias/X_usage_bias_times_dense1.npy',bias)
weight = model.get_weights()[2]
bias = model.get_weights()[3]
np.save('./weight_bias/X_usage_weight_times_dense2.npy',weight)
np.save('./weight_bias/X_usage_bias_times_dense2.npy',bias)
weight = model.get_weights()[4]
bias = model.get_weights()[5]
np.save('./weight_bias/X_usage_weight_times_dense3.npy',weight)
np.save('./weight_bias/X_usage_bias_times_dense3.npy',bias)

In [ ]:
from keras.models import load_model
#保存X_usage权重
model = load_model('./weight_bias/best_epoch_model_usage.hdf5')
weight = model.get_weights()[0]
bias = model.get_weights()[1]
weight = weight[:X_usage.shape[1],:]
np.save('./weight_bias/X_usage_weight_dense1.npy',weight)
np.save('./weight_bias/X_usage_bias_dense1.npy',bias)
weight = model.get_weights()[2]
bias = model.get_weights()[3]
np.save('./weight_bias/X_usage_weight_dense2.npy',weight)
np.save('./weight_bias/X_usage_bias_dense2.npy',bias)
weight = model.get_weights()[4]
bias = model.get_weights()[5]
np.save('./weight_bias/X_usage_weight_dense3.npy',weight)
np.save('./weight_bias/X_usage_bias_dense3.npy',bias)

In [38]:
# evaluate the model
age_test = pd.read_csv('../age_test.csv', header = None, names = ['uId'])
print("# Final prediction")
scores = model.predict(test_x)
test_label = targetencoder.inverse_transform(np.argmax(scores, axis=1))
result = pd.DataFrame()
result['id'] = age_test.uId
result['label'] = test_label.astype(int)
result.to_csv('./submission.csv', index=False)
print("Done")

# Final prediction
Done
